## 导入数据到hive
   
先建数据库 cda2

/opt/hive/bin/beeline -u jdbc:hive2://localhost:10000

0: jdbc:hive2://localhost:10000>  create database if not exists shop;

建立user表

CREATE TABLE shop.users (user_id INT ,item_id INT ,cat_id INT ,merchant_id INT ,brand_id INT ,month INT ,day INT ,action INT ,age_range INT ,gender INT ,province STRING )   ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' STORED AS TEXTFILE tblproperties("skip.header.line.count"="1");

中文编码
ALTER TABLE shop.users SET SERDEPROPERTIES ('serialization.encoding'='GBK');

导入数据

LOAD DATA LOCAL INPATH '/opt/hive/data/users.csv' OVERWRITE INTO TABLE shop.users;

查询前5行数据

select * from shop.users limit 5;


## 查询users.csv 表结构


desc shop.users;
+--------------+------------+----------+
|   col_name   | data_type  | comment  |
+--------------+------------+----------+
| user_id      | int        |          |
| item_id      | int        |          |
| cat_id       | int        |          |
| merchant_id  | int        |          |
| brand_id     | int        |          |
| month        | int        |          |
| day          | int        |          |
| action       | int        |          |
| age_range    | int        |          |
| gender       | int        |          |
| province     | string     |          |
+--------------+------------+----------+
 
## 查表的详细信息

desc formatted shop.users;

## 查看数据总量

select count(*) from shop.users;

## 使用distinct方法查询不重复数据数量

select count(distinct user_id ,item_id,cat_id ,merchant_id,brand_id,month,day ,action,age_range,gender) from shop.users;

如果能用group by的就尽量使用group by，因为group by性能更好

## 查询双十一当天发生购买行为的顾客数量

Select count(distinct(user_id))  from shop.users where month=11 and day=11  and action =1;
+-------+
|  _c0  |
+-------+
| 9290  |
+-------+


## 用spark写入hive


In [8]:
from pyspark import SparkConf, SparkContext 
from pyspark.sql import HiveContext 

spark = SparkSession\
        .builder \
        .appName("Spark to Hive") \
        .master("spark://master:7077")\
        .config("hive.metastore.uris", "thrift://129.211.84.214:9083") \
        .enableHiveSupport()\
        .getOrCreate()
        
#sparkSession.read.option("header", "true").csv("src/main/resources/scala.csv")
df = spark.read.csv("data/train.csv")  # 把csv读成dataframe
## 其他参数
# schema – an optional pyspark.sql.types.StructType for the input schema.
# header：默认值是false。就是把第一行当做数据，改为false，第一行就变为字段；
# sep：默认情况下，CSV是使用英文逗号分隔的,其他分隔符号可修改此选项;

# 拼接一个字段类型字符串
str_s = 'label String,'
for i in range(len(df.columns)-1):
    str_s += 'pixel%s INT,' % i
# 拼接SQL语句
sql_str = "create table shop.train ({})".format(str_s[:-1])

spark.sql(sql_str) #　执行ＳＱＬ
　
df.write.insertInto('shop.train', overwrite=False)  # 将dataframe写入到指定hive表